# Library

## Install

```cmd
pip install transformers
```

In [1]:
!pip install transformers tf-keras sacremoses accelerate datasets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   --------------------- ------------------ 5.2/9.9 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 36.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 91.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   --------------------------------------- 897.5/897.5 kB 39.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 141.5 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Load

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    BertConfig, BertModel, BertForSequenceClassification, BertForMaskedLM,
    Trainer, TrainingArguments,
)

# Transformers

documentation: https://huggingface.co/docs/transformers/ko/index <br>
pre-trained model list: https://huggingface.co/models

<br>

Hugging Face의 Transformers 라이브러리는 자연어 처리(NLP) 및 기타 언어 관련 작업을 위한 강력한 도구

## 주요 기능

다양한 모델 지원: BERT, GPT 등 여러 가지 인기 있는 모델을 포함 <br>
사전 훈련된 모델: NLP 작업에 맞게 사전 훈련된 다양한 모델을 쉽게 다운로드하고 사용 가능 <br>
간편한 API: 사용자 친화적인 API를 제공하여 복잡한 작업을 간단하게 처리 가능 <br>
PyTorch 및 TensorFlow 지원: 두 가지 주요 딥러닝 프레임워크를 지원하여 유연성을 제공 <br>
데이터셋과의 통합: datasets 라이브러리와 통합되어 쉽게 데이터셋을 로드 가능 <br>

## 주요 구성 요소

Tokenizer: 텍스트 데이터를 모델에 입력할 수 있는 형식으로 변환합니다. 토크나이저는 입력 문장을 토큰으로 나누고, 이를 숫자 인덱스로 변환 <br>
Model: 다양한 NLP 작업에 사용할 수 있는 사전 훈련된 모델 <br>
Trainer API: 모델을 쉽게 훈련하고 평가할 수 있는 고수준 API. 훈련 인자, 데이터셋 등을 설정하여 간편하게 사용 가능 <br>

### Tokenizer

사용할 tokenizer 로드 <br>
**사용하는 모델에 맞는 tokenizer 선택** <br>

<br>

<span style="font-size: 16pt;"> 사용 방법 </span>

> ```python
> tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
> inputs = tokenizer('Hello world', return_tensors='pt') # 단일 sequence tokenize
> inputs = tokenizer(['Hello, how are you?', 'I am fine, thank you.', 'What about you?'], padding=True, truncation=True, return_tensors='pt')  # 복수 sequence tokenize
> # 두 문장 tokenize
> inputs = tokenizer.encode_plus(
>     question,                       # 첫 번째 텍스트 (질문)
>     answer,                         # 두 번째 텍스트 (답변)
>     add_special_tokens=True,       
>     return_tensors='pt'           
> )
> tokenizer.decode(inputs.get('input_ids'))[0]  # special token을 포함한 원래 문장으로 출력
> tokenizer.convert_ids_to_tokens(inputs.get('input_ids'))[0]   # token으로 출력
> ```

arguments: 
- padding: padding을 어떤 기준에 맞춰 할 건지 결정
- truncation: 최대 길이를 넘어가는 token을 제거할 건지 결정
    - True or longest: batch 내 가장 긴 길이에 맞게 padding
    - max_length: argument max_length 또는 max_length argument에 정의된 길이에 맞게 padding
- max_length: 최대 길이 정의
- return_tensors: return type을 tensor로 반환

In [4]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [30]:
encoded = tokenizer(
    ['hello, world', 'how are uou?', 'i\'m fine. and you?'],
    max_length=8,
    padding=True,
    truncation=True,
    return_tensors='pt',
)
encoded

{'input_ids': tensor([[ 101, 7592, 1010, 2088,  102,    0,    0,    0],
        [ 101, 2129, 2024, 1057, 7140, 1029,  102,    0],
        [ 101, 1045, 1005, 1049, 2986, 1012, 1998,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
tokenizer.convert_ids_to_tokens(encoded.get('input_ids')[1])

['[CLS]', 'how', 'are', 'u', '##ou', '?', '[SEP]', '[PAD]']

In [36]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-uncased')
tokenizer.convert_ids_to_tokens(tokenizer('안녕하세요. 오늘은 bert를 진행하는 시간입니다').get('input_ids'))

['[CLS]',
 'ᄋ',
 '##ᅡᆫ',
 '##녀',
 '##ᆼ',
 '##하',
 '##세',
 '##요',
 '.',
 'ᄋ',
 '##ᅩ',
 '##ᄂ',
 '##ᅳᆯ',
 '##은',
 'bert',
 '##를',
 '진',
 '##행',
 '##하는',
 '시',
 '##간이',
 '##ᆸ니다',
 '[SEP]']

### Model

모델을 로드하는 부분 <br>
from_pretrained를 통해 사전에 학습된 weight를 불러올 수 있음 <br>
ModelConfig를 통해 model에서 사용할 세부적인 parameter 조절 가능 <br>

<br>

모델에 따라 사전에 정의된 모델에 약간의 차이는 존재 <br>
ex) BERT <br>
- BertModel
- BertForPreTraining
- BertLMHeadModel
- BertForMaskedLM
- BertForNextSentencePrediction
- BertForSequenceClassification
- BertForMultipleChoice
- BertForTokenClassification
- BertForQuestionAnswering

<br>

<span style="font-size: 16pt;"> 사용 방법 </span>

> ```python
> config = BertConfig()
> model = BertModel(config) # 정의된 config를 이용하여 모델 생성
> 
> model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-uncased") # classication을 위한 BERT model과 weight load
> outputs = model(**inputs, output_hidden_states=True)    # hidden states 출력
> last_hidden_state = outputs[-1]    # last hidden states
> ```

In [49]:
config = BertConfig()
config.update({
    'hidden_size': 1024,
    'intermediate_size': 4096,
    'num_hidden_layers': 24,
    'num_attention_heads': 16,
})
bert = BertModel(config)
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [55]:
bert(**tokenizer('hello world', return_tensors='pt')).last_hidden_state

torch.Size([1, 4, 1024])

In [59]:
bert(**tokenizer('hello world', return_tensors='pt')).pooler_output.shape

torch.Size([1, 1024])

In [64]:
len(bert(**tokenizer('hello world', return_tensors='pt'), output_hidden_states=True).hidden_states)

25

### Trainer

모델 학습을 위한 객체 <br>
TrainingArguments를 통해 세부적인 학습 내용 조절 가능 <br>
Trainer를 통해 객체 생성 후 학습 <br>

<br>

<span style="font-size: 16pt;"> 사용 방법 </span>

> ```python
> # 훈련 인자 설정
> training_args = TrainingArguments(
>     output_dir='./results',            # 출력 경로
>     evaluation_strategy="epoch",       # 평가 전략
>     learning_rate=2e-5,                # 학습률
>     per_device_train_batch_size=8,     # 배치 크기
>     per_device_eval_batch_size=8,      # 검증 배치 크기
>     num_train_epochs=3,                # 에폭 수
>     weight_decay=0.01,                 # 가중치 감쇠
> )
> 
> def compute_metrics(outputs):
>     preds = outputs.predictions.argmax(-1)
>     precision, recall, f1, _ = precision_recall_fscore_support(outputs.label_ids, preds, average='binary')
>     acc = accuracy_score(outputs.label_ids, preds)
>     return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}
> 
> # Trainer 객체 생성
> trainer = Trainer(
>     model=model,
>     args=training_args,
>     train_dataset=encoded_dataset["train"],
>     eval_dataset=encoded_dataset["test"],
>     compute_metrics=compute_metrics      # 평가 메트릭, dictionary 형태로 return
> )
> # 모델 학습
> trainer.train()
>
> # 모델 평가
> trainer.eval()
> ```


In [69]:
bert = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# train_data, eval_data
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

def compute_metrics(outputs):
    pred = outputs.prediction.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(outputs.label_ids, pred, average='binary')
    acc = accuracy_score(outputs.label_ids, pred)

    return {'acc': acc, 'precision': precision, 'recall': recall, 'f1': f1}

trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.eval()